In [ ]:
import os

colab = True

if colab:
  from google.colab import drive
  drive.mount('/content/drive/')
  dataset_path = "/content/drive/My Drive"

else:
  dataset_path = "/mnt/c/Users/valent1ne/Documents/Dataset/Segmentation "

In [ ]:
from natsort import natsorted

set_two_rgb_path = os.path.join(dataset_path, "Jonathans Data/RgbImages")
set_two_masks_path = os.path.join(dataset_path, "Jonathans Data/PlantMask")
set_two_filenames = natsorted(os.listdir(set_two_rgb_path))
set_two_masknames = natsorted(os.listdir(set_two_masks_path))
len(list(set(set_two_filenames))),  len(list(set(set_two_masknames)))

In [3]:
set_two_masknames = [x for i, x in enumerate(set_two_masknames) if i%2==0]

In [ ]:
len(set_two_masknames)

In [5]:
input_images = set_two_filenames
mask_images = set_two_masknames

In [6]:
interpolation_method = "bilinear"

In [7]:
from keras.preprocessing import image
import numpy as np

# img = image.load_img(sample_image_path)
# array = image.img_to_array(img)

In [ ]:
len(input_images)

In [ ]:
from sklearn.utils import shuffle

x = 123
# print(input_images[x])
# print(mask_images[x])

shuffled_x, shuffled_y = shuffle(input_images, mask_images)
print(shuffled_x[x])
print(shuffled_y[x])

In [ ]:
for i, v in enumerate(shuffled_x):
  if i < 2:
    print(f"{v}\n{shuffled_y[i]}\n\n")

In [ ]:
shuffled_y[163]

In [ ]:
os.listdir(os.path.join(dataset_path, "Jonathans Data"))

In [ ]:
# THIS IS COPY OF THE IMAGE PROCESSING CODE JUST FOR TESTING. ONLY RUNS FIRST 5 IMAGES


from keras.preprocessing.image import smart_resize
if colab:
  !pip install -q plantcv==3.13.4
  !pip install scipy
from plantcv import plantcv as pcv
import cv2
from scipy.signal import medfilt2d
import numpy as np
import matplotlib.pyplot as plt

x_train = np.ndarray((len(shuffled_x), 512, 512, 1), dtype=np.uint8)
y_train = np.ndarray((len(shuffled_y), 512, 512, 1), dtype=np.uint8)
pcv.params.debug = 'plot'                     # Plot debug images to the notebook

for index, datapoint in enumerate(shuffled_x):
    if index < 5:
        # Load image
        img = image.load_img(os.path.join(dataset_path, "Jonathans Data", "RgbImages", datapoint))
  
        #Turn into an array
        img_array = image.img_to_array(img)
        
        # Resize image using bilinear interpolation
        img_array = cv2.resize(img_array, dsize=(512, 512))

        r = img_array[:,:,0:1]
        g = img_array[:,:,1:2]
        b = img_array[:,:,2:3]

        img_array = 2*g - r - b

        # img_array = medfilt2d(img_array)

        # Normalize array to be between 0-1 (default is from 0 to 255, cause RGB)
        # img_array = img_array / 255

        # plt.imshow(np.squeeze(medfilt2d(img_array)))


        # Select "A" channel of LAB color scheme
        # img_array = pcv.rgb2gray_lab(rgb_img=img_array, channel="a")

        # Resize array to be correct shape for tensor
        # img_array = np.expand_dims(img_array, axis=2)
       
        # Add resulting array to the list of arrays
        x_train[index] = img_array
        
        mask = image.load_img(os.path.join(dataset_path, "Jonathans Data", "PlantMask", shuffled_y[index]), color_mode="grayscale") 
        mask_array = image.img_to_array(mask)
        mask_array = cv2.resize(mask_array, dsize=(512, 512))
        mask_array = mask_array > 0
        mask_array = mask_array.astype(np.uint8)   
        # print(mask_array.shape)
        # plt.imshow(mask_array)
        # print(np.unique(mask_array))
        
        
        # y_train[index] = nexpand_dims(mask_array, axis=2)
        
        print(f"Processed {index + 1} out of {len(input_images)} images")

In [ ]:
from keras.preprocessing.image import smart_resize
from plantcv import plantcv as pcv
import cv2
import numpy as np
import matplotlib.pyplot as plt
from scipy.signal import medfilt2d

x_values = np.ndarray((len(shuffled_x), 512, 512, 1), dtype=np.uint8)
y_values = np.ndarray((len(shuffled_y), 512, 512, 1), dtype=np.uint8)
pcv.params.debug = None                     # Plot debug images to the notebook

for index, datapoint in enumerate(shuffled_x):
    # if index < 5:
        # Load image
        img = image.load_img(os.path.join(dataset_path, "Jonathans Data", "RgbImages", datapoint))
  
        #Turn into an array
        img_array = image.img_to_array(img)
        
        # Resize image using bilinear interpolation
        img_array = cv2.resize(img_array, dsize=(512, 512))
 
        r = img_array[:,:,0:1]
        g = img_array[:,:,1:2]
        b = img_array[:,:,2:3]

        img_array = 2*g - r - b
        
        # Normalize array to be between 0-1 (default is from 0 to 255, cause RGB)
        # img_array = img_array / 255
        
        # Select "A" channel of LAB color scheme
        # img_array = pcv.rgb2gray_lab(rgb_img=img_array, channel="A")


        # Resize array to be correct shape for tensor
        # img_array = np.expand_dims(img_array, axis=2)
       
        # Add resulting array to the list of arrays
        x_values[index] = img_array
        
        mask = image.load_img(os.path.join(dataset_path, "Jonathans Data", "PlantMask", shuffled_y[index]), color_mode="grayscale") 
        mask_array = image.img_to_array(mask)
        mask_array = cv2.resize(mask_array, dsize=(512, 512))
        mask_array = mask_array > 0
        mask_array = mask_array.astype(np.uint8)        
        mask_array = np.expand_dims(mask_array, axis=2)
        
        y_values[index] = mask_array
        
        if index % 50 == True:
            print(f"Processed {index + 1} out of {len(input_images)} images")

In [15]:
test_train_ratio = 0.9

x_train = x_values[:round(len(x_values) * test_train_ratio)]
x_test = x_values[round(len(x_values) * test_train_ratio):]
y_train = y_values[:round(len(y_values) * test_train_ratio)]
y_test = y_values[round(len(y_values) * test_train_ratio):]

In [ ]:
print(f"x_train: {len(x_train)} x_test: {len(x_test)} y_train: {len(y_train)} y_test: {len(y_test)}")

In [ ]:
shuffled_x[163]

In [24]:
from tensorflow.keras import Input, Model
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Dropout, UpSampling2D, concatenate
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.metrics import MeanIoU


def unet(pretrained_weights=None, input_size=(512, 512, 1)):
    inputs = Input(input_size)
    conv1 = Conv2D(64, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(inputs)
    conv1 = Dropout(0.1)(conv1)
    conv1 = Conv2D(64, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv1)
    pool1 = MaxPooling2D(pool_size=(2, 2))(conv1)

    conv2 = Conv2D(128, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(pool1)
    conv2 = Dropout(0.1)(conv2)
    conv2 = Conv2D(128, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv2)
    pool2 = MaxPooling2D(pool_size=(2, 2))(conv2)
    
    conv3 = Conv2D(256, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(pool2)
    conv3 = Dropout(0.2)(conv3)
    conv3 = Conv2D(256, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv3)
    pool3 = MaxPooling2D(pool_size=(2, 2))(conv3)
    
    conv4 = Conv2D(512, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(pool3)
    conv4 = Conv2D(512, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv4)
    drop4 = Dropout(0.3)(conv4)
    pool4 = MaxPooling2D(pool_size=(2, 2))(drop4)

    conv5 = Conv2D(1024, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(pool4)
    conv5 = Conv2D(1024, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv5)
    drop5 = Dropout(0.3)(conv5)

    up6 = Conv2D(512, 2, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(UpSampling2D(size = (2,2))(drop5))
    merge6 = concatenate([drop4,up6], axis = 3)
    conv6 = Conv2D(512, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(merge6)
    conv6 = Dropout(0.2)(conv6)
    conv6 = Conv2D(512, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv6)

    up7 = Conv2D(256, 2, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(UpSampling2D(size = (2,2))(conv6))
    merge7 = concatenate([conv3,up7], axis = 3)
    conv7 = Conv2D(256, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(merge7)
    conv7 = Dropout(0.2)(conv7)
    conv7 = Conv2D(256, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv7)

    up8 = Conv2D(128, 2, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(UpSampling2D(size = (2,2))(conv7))
    merge8 = concatenate([conv2,up8], axis = 3)
    conv8 = Conv2D(128, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(merge8)
    conv8 = Dropout(0.2)(conv8)
    conv8 = Conv2D(128, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv8)

    up9 = Conv2D(64, 2, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(UpSampling2D(size = (2,2))(conv8))
    merge9 = concatenate([conv1,up9], axis = 3)
    conv9 = Conv2D(64, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(merge9)
    conv9 = Conv2D(64, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv9)
    conv9 = Conv2D(2, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv9)

    conv10 = Conv2D(1, 1, activation = 'sigmoid')(conv9)

    model = Model(inputs=inputs, outputs=conv10)
    # model.compile(optimizer="adam", loss="binary_crossentropy", metrics=[MeanIoU(num_classes=2)])
    model.compile(optimizer="adam", loss='binary_crossentropy', metrics=['accuracy', MeanIoU(num_classes=2)])


    if pretrained_weights:
        model.load_weights(pretrained_weights)

    return model

In [ ]:
model = unet()
model.summary()

In [26]:
from tensorflow.keras.callbacks import ModelCheckpoint, TensorBoard, ReduceLROnPlateau, EarlyStopping


model_name = 'four_epochs.h5'
log_dir = "logs/000"
logging = TensorBoard(log_dir=log_dir)
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.1, patience=3, verbose=1)
early_stopping = EarlyStopping(monitor='val_loss', min_delta=0, patience=10, verbose=1)

In [ ]:
model_checkpoint = ModelCheckpoint('unet.h5', monitor='val_loss', verbose=1, save_best_only=True)
model.fit(x_train, y_train, batch_size=4, epochs=5, verbose=1,
                      validation_split=0.2, shuffle=True,
                      callbacks=[logging, model_checkpoint, reduce_lr, early_stopping])
model.save(model_name)

In [28]:
pred = model.predict(x_test, batch_size=4) 
# pred = np.argmax(pred, axis = 1)
# label = np.argmax(y_test,axis = 1)

In [ ]:
for x in x_test:
  sample_prediction = np.squeeze(x)
  print(len(np.unique(sample_prediction)))

In [ ]:
score = model.evaluate(x_test, y_test, batch_size=4, verbose=1)
print('Test loss:', score[0]) 
print('Test accuracy:', score[1])

# Trying to predict on jason's dataet

In [ ]:
import os

colab = True

if colab:
  from google.colab import drive
  drive.mount('/content/drive')
  dataset_path = "/content/drive/My Drive/Segmentation - 01 reduced"

else:
  dataset_path = "/mnt/c/Users/valent1ne/Documents/Dataset/Segmentation - 01"



In [ ]:
sample_image_path = os.path.join(dataset_path, os.listdir(dataset_path)[123])
sample_image_path

In [ ]:
from natsort import natsorted

natsorted(os.listdir(dataset_path))[:10]

In [37]:
interpolation_method = "bilinear"

In [38]:
from keras.preprocessing import image
import numpy as np

img = image.load_img(sample_image_path)
array = image.img_to_array(img)

In [39]:
from natsort import natsorted
dataset_filenames = natsorted(os.listdir(dataset_path))
input_images = [x for x in dataset_filenames if "FEC_" in x]
mask_images =  [x for x in dataset_filenames if "Coloring_" in x]

In [ ]:
len(input_images)

In [ ]:
from sklearn.utils import shuffle

x = 234
# print(input_images[x])
# print(mask_images[x])

shuffled_x, shuffled_y = shuffle(input_images, mask_images)
print(shuffled_x[x])
print(shuffled_y[x])

In [ ]:
for i, v in enumerate(shuffled_x):
  if i < 2:
    print(f"{v}\n{shuffled_y[i]}\n\n")

In [ ]:
# THIS IS COPY OF THE IMAGE PROCESSING CODE JUST FOR TESTING. ONLY RUNS FIRST 5 IMAGES


from keras.preprocessing.image import smart_resize
if colab:
  !pip install -q plantcv==3.13.4
from plantcv import plantcv as pcv
import cv2
import numpy as np
import matplotlib.pyplot as plt

x_train = np.ndarray((len(shuffled_x), 512, 512, 1), dtype=np.uint8)
y_train = np.ndarray((len(shuffled_y), 512, 512, 1), dtype=np.uint8)
# pcv.params.debug = 'plot'                     # Plot debug images to the notebook
pcv.params.debug = None                     # Plot debug images to the notebook


for index, datapoint in enumerate(shuffled_x):
    if index < 1:
        # Load image
        img = image.load_img(os.path.join(dataset_path, datapoint))
  
        #Turn into an array
        img_array = image.img_to_array(img)
        
        # Resize image using bilinear interpolation
        img_array = cv2.resize(img_array, dsize=(512, 512))
        
        # Normalize array to be between 0-1 (default is from 0 to 255, cause RGB)
        img_array = img_array / 255
        
        # Select "A" channel of LAB color scheme
        img_array = pcv.rgb2gray_lab(rgb_img=img_array, channel="A")

        # Resize array to be correct shape for tensor
        img_array = np.expand_dims(img_array, axis=2)
       
        # Add resulting array to the list of arrays
        x_train[index] = img_array
        
        mask = image.load_img(os.path.join(dataset_path, shuffled_y[index]), color_mode="grayscale") 
        mask_array = image.img_to_array(mask)
        mask_array = cv2.resize(mask_array, dsize=(512, 512))
        temp_arr = mask_array > 0
        mask_array = temp_arr.astype(np.uint8)
        # mask_array = mask_array 
        # np.savetxt("mask.txt", mask_array)
        print(np.unique(mask_array))
        plt.imshow(mask_array)
        
        # y_train[index] = nexpand_dims(mask_array, axis=2)
        
        print(f"Processed {index + 1} out of {len(input_images)} images")

In [ ]:
from keras.preprocessing.image import smart_resize
from plantcv import plantcv as pcv
import cv2
import numpy as np
import matplotlib.pyplot as plt

x_values = np.ndarray((len(shuffled_x), 512, 512, 1), dtype=np.uint8)
y_values = np.ndarray((len(shuffled_y), 512, 512, 1), dtype=np.uint8)
pcv.params.debug = None                     # Plot debug images to the notebook

for index, datapoint in enumerate(shuffled_x):
    # if index < 5:
        # Load image
        img = image.load_img(os.path.join(dataset_path, datapoint))
  
        #Turn into an array
        img_array = image.img_to_array(img)
        
        # Resize image using bilinear interpolation
        img_array = cv2.resize(img_array, dsize=(512, 512))
        
        # Normalize array to be between 0-1 (default is from 0 to 255, cause RGB)
        img_array = img_array / 255
        
        # Select "A" channel of LAB color scheme
        img_array = pcv.rgb2gray_lab(rgb_img=img_array, channel="A")

        # Resize array to be correct shape for tensor
        img_array = np.expand_dims(img_array, axis=2)
       
        # Add resulting array to the list of arrays
        x_values[index] = img_array
        
        mask = image.load_img(os.path.join(dataset_path, shuffled_y[index]), color_mode="grayscale") 
        mask_array = image.img_to_array(mask)
        mask_array = cv2.resize(mask_array, dsize=(512, 512))
        # mask_array = mask_array / 255
        mask_array = mask_array > 0
        mask_array = mask_array.astype(np.uint8)

        mask_array = np.expand_dims(mask_array, axis=2)
        
        y_values[index] = mask_array
        
        if index % 50 == True:
            print(f"Processed {index + 1} out of {len(input_images)} images")

In [ ]:
score = model.evaluate(x_values, y_values, batch_size=4, verbose=1)
print('Test loss:', score[0]) 
print('Test accuracy:', score[1])



In [ ]:
model.predict

In [47]:
from keras import backend as K
def iou_coef(y_true, y_pred, smooth=1):
  intersection = K.sum(K.abs(y_true * y_pred), axis=[1,2,3])
  union = K.sum(y_true,[1,2,3])+K.sum(y_pred,[1,2,3])-intersection
  iou = K.mean((intersection + smooth) / (union + smooth), axis=0)
  return iou